测试三重过滤分析系统报告

In [1]:
import backtrader as bt
import akshare as ak
import os
import logging
from logging.handlers import TimedRotatingFileHandler
from datetime import datetime
from my_backtrader.tts import TripleScreenTradingSystem

In [2]:
def init_logging():
    """全局日志配置（在策略初始化前调用）"""
    log_format = '%(asctime)s [%(levelname)s] %(message)s'
    log_file = 'logs/futures.log'
    
    # 创建日志目录
    os.makedirs('logs', exist_ok=True)
    
    # 主日志配置
    logging.basicConfig(
        level=logging.INFO,
        format=log_format,
        handlers=[
            TimedRotatingFileHandler(
                log_file, 
                when='D',  # 按天切割
                backupCount=7,
                encoding='utf-8'
            ),
            logging.StreamHandler()
        ]
    )
    
init_logging()

In [3]:
import pandas as pd


def futures_main_weekly_sina(
    symbol: str = "V0",
    start_date: str = "19900101",
    end_date: str = "22220101",
) -> pd.DataFrame:
    """
    新浪财经-期货-主力连续周线数据
    基于日线数据聚合计算，或直接调用周线接口
    """
    # 1. 获取日线数据后聚合为周线（通用方法）
    daily_df = ak.futures_main_sina(symbol=symbol, start_date=start_date, end_date=end_date)
    daily_df["日期"] = pd.to_datetime(daily_df["日期"])
    daily_df.set_index("日期", inplace=True)
    
    # 聚合逻辑：每周一至周五为一周，取首开、最高、最低、尾收、成交量和持仓量总和
    weekly_df = daily_df.resample("W-FRI").agg({
        "开盘价": "first",
        "最高价": "max",
        "最低价": "min",
        "收盘价": "last",
        "成交量": "sum",
        "持仓量": "sum",
        "动态结算价": "last"
    }).dropna()
    
    # 2. 日期筛选
    weekly_df = weekly_df[(weekly_df.index >= pd.to_datetime(start_date)) & 
                         (weekly_df.index <= pd.to_datetime(end_date))]
    weekly_df.reset_index(inplace=True)
    weekly_df.rename(columns={"index": "日期"}, inplace=True)
    return weekly_df

# 示例调用
weekly_data = futures_main_weekly_sina(symbol="V0", start_date="20240101", end_date="20251231")
print(weekly_data)

           日期     开盘价     最高价     最低价     收盘价      成交量      持仓量   动态结算价
0  2024-01-05  5880.0  5953.0  5814.0  5824.0  2755934  3516450  5843.0
1  2024-01-12  5840.0  5867.0  5754.0  5827.0  3376627  4592776  5825.0
2  2024-01-19  5835.0  5904.0  5803.0  5872.0  3078001  4140989  5856.0
3  2024-01-26  5880.0  5970.0  5823.0  5893.0  3442483  3762404  5913.0
4  2024-02-02  5898.0  5954.0  5783.0  5812.0  3252193  3855382  5818.0
..        ...     ...     ...     ...     ...      ...      ...     ...
90 2025-10-03  4894.0  4918.0  4838.0  4839.0  1246327  2147720  4858.0
91 2025-10-10  4825.0  4825.0  4719.0  4735.0  1396656  2361602  4754.0
92 2025-10-17  4730.0  4736.0  4644.0  4688.0  3456442  6243173  4706.0
93 2025-10-24  4696.0  4749.0  4670.0  4708.0  3067461  6033425  4716.0
94 2025-10-31  4708.0  4817.0  4687.0  4701.0  4286043  6040333  4729.0

[95 rows x 8 columns]


In [4]:
class FuturesDataFeed(bt.feeds.PandasData):
    """适配期货数据的Backtrader数据源"""
    params = (
        ('datetime', '日期'),
        ('open', '开盘价'),
        ('high', '最高价'),
        ('low', '最低价'),
        ('close', '收盘价'),
        ('volume', '成交量'),
        ('openinterest', '持仓量'),
    )

In [5]:
symbol_name = "红枣连续"
symbol = 'CJ0'

daily_df = ak.futures_main_sina(symbol=symbol, start_date='20150101')
weekly_df = futures_main_weekly_sina(symbol=symbol, start_date="20150101")

In [6]:
print(daily_df.head())
print(daily_df.tail())

           日期   开盘价   最高价   最低价   收盘价     成交量    持仓量  动态结算价
0  2019-05-06  8820  8900  8620  8665  349204  57284   8745
1  2019-05-07  8710  8720  8545  8615  301676  70904   8620
2  2019-05-08  8620  8710  8595  8635  234442  71964   8650
3  2019-05-09  8635  8690  8580  8620  203022  70710   8640
4  2019-05-10  8655  8750  8620  8670  188466  65566   8680
              日期    开盘价    最高价    最低价    收盘价     成交量     持仓量  动态结算价
1571  2025-10-27  10780  10875  10375  10390  377382  185516  10595
1572  2025-10-28  10420  10530  10310  10445  265154  185560  10415
1573  2025-10-29  10460  10565  10375  10495  199932  184472  10485
1574  2025-10-30  10500  10525  10175  10225  324621  188572  10295
1575  2025-10-31  10285  10375  10120  10145  245521  180474  10260


In [7]:
print(weekly_df.head())
print(weekly_df.tail())

          日期      开盘价      最高价      最低价      收盘价      成交量     持仓量    动态结算价
0 2019-05-10   8820.0   8900.0   8545.0   8670.0  1276810  336428   8680.0
1 2019-05-17   8690.0   9890.0   8670.0   9755.0  2755091  553086   9715.0
2 2019-05-24   9790.0  10730.0   9655.0  10720.0  5830458  728590  10490.0
3 2019-05-31  10600.0  10935.0  10295.0  10485.0  7067540  777618  10440.0
4 2019-06-07  10435.0  10560.0   9625.0   9750.0  3277814  535030   9770.0
            日期      开盘价      最高价      最低价      收盘价      成交量     持仓量    动态结算价
328 2025-10-03  11270.0  11270.0  10660.0  10820.0   425451  287415  10790.0
329 2025-10-10  10880.0  11230.0  10765.0  11145.0   452218  305577  11110.0
330 2025-10-17  11165.0  11480.0  11005.0  11420.0   831547  826905  11405.0
331 2025-10-24  11450.0  11470.0  10735.0  10750.0  1062156  927827  10960.0
332 2025-10-31  10780.0  10875.0  10120.0  10145.0  1412610  924594  10260.0


In [8]:
daily_df['日期'] = pd.to_datetime(daily_df['日期'])
daily_df = daily_df.sort_values('日期')

weekly_df['日期'] = pd.to_datetime(weekly_df['日期'])
weekly_df = weekly_df.sort_values('日期')

daily_data = FuturesDataFeed(dataname=daily_df)
weekly_data = FuturesDataFeed(dataname=weekly_df)

In [9]:
def run_backtrader_strategy():
    cerebro = bt.Cerebro()

    cerebro.adddata(daily_data, name=symbol_name)
    cerebro.adddata(weekly_data, name=symbol_name)
    cerebro.addstrategy(TripleScreenTradingSystem, symbol = 'V0')
    return cerebro.run()


In [10]:
# 每次调用都会创建新的 cerebro
# results = run_backtrader_strategy()

In [11]:
def run_backtest():
    cerebro = bt.Cerebro()
    
    # 添加策略
    cerebro.addstrategy(TripleScreenTradingSystem)
    
    # 设置初始资金
    cerebro.broker.setcash(100000.0)
    
    # 设置手续费 - 期货手续费较低
    cerebro.broker.setcommission(commission=0.0005)  # 0.05%
    cerebro.adddata(daily_data, name=f"{symbol_name}")
    cerebro.adddata(weekly_data, name=f"{symbol_name}_WEEKLY")

    
    print('初始资金: %.2f' % cerebro.broker.getvalue())
    
    # 运行回测
    cerebro.run()
    
    # 绘制图表
    cerebro.plot()
    
    print('最终资金: %.2f' % cerebro.broker.getvalue())

In [12]:
run_backtest()

初始资金: 100000.00


2025-11-03 00:10:27,916 [INFO] 2020-01-03: 发出做多订单: 价格=10905.00
2025-11-03 00:10:27,917 [INFO] 2020-01-06: 🔍 订单完成检查: size=1, position=1
2025-11-03 00:10:27,917 [INFO] 2020-01-06: ✅ 多头开仓完成: 价格=10880.00, entry_bar=170, entry_price=10880.00
2025-11-03 00:10:27,918 [INFO] 2020-01-06: 📋 买入开多仓完成: 价格=10880.00, 成本=10880.00, 手续费=5.44
2025-11-03 00:10:27,919 [INFO] 2020-01-07: 多头平仓，周趋势反转
2025-11-03 00:10:27,920 [INFO] 2020-01-08: 🔍 订单完成检查: size=-1, position=None
2025-11-03 00:10:27,920 [INFO] 2020-01-08: ⚠️ 跳过开仓设置: position_direction=多头, entry_price=10880.0
2025-11-03 00:10:27,921 [INFO] 2020-01-08: 📋 卖出平多仓完成: 价格=10490.00, 成本=10880.00, 手续费=5.25
2025-11-03 00:10:27,921 [INFO] 2020-01-08:   系统毛利润: -390.00
2025-11-03 00:10:27,921 [INFO] 2020-01-08:   系统手续费: 10.69
2025-11-03 00:10:27,922 [INFO] 2020-01-08:   系统净利润: -400.69
2025-11-03 00:10:27,922 [INFO] 2020-01-08:   收益率: -1.03%
2025-11-03 00:10:27,923 [INFO] 2020-01-08: 📊 平仓: 毛利润=-390.00, 净利润=-400.69, 收益率=-1.03%
2025-11-03 00:10:27,923 [INFO] 2020-0

日期: None 当前信号=1
趋势转变! 时间: 2019-12-27, 价格: 10875.00, 类型: 上涨
日期: 2019-12-27 当前信号=1
日期: 2019-12-27 当前信号=1
日期: 2019-12-27 当前信号=1
日期: 2019-12-27 当前信号=1
日期: 2019-12-27 当前信号=1
日期: 2019-12-27 当前信号=1
日期: 2019-12-27 当前信号=1
日期: 2019-12-27 当前信号=1
日期: 2019-12-27 当前信号=1
日期: 2019-12-27 当前信号=-1
趋势转变! 时间: 2020-03-13, 价格: 10360.00, 类型: 下跌
日期: 2020-03-13 当前信号=-1
日期: 2020-03-13 当前信号=-1
日期: 2020-03-13 当前信号=-1
日期: 2020-03-13 当前信号=-1
日期: 2020-03-13 当前信号=-1
日期: 2020-03-13 当前信号=-1
日期: 2020-03-13 当前信号=-1
日期: 2020-03-13 当前信号=-1
日期: 2020-03-13 当前信号=-1
日期: 2020-03-13 当前信号=-1
日期: 2020-03-13 当前信号=-1
日期: 2020-03-13 当前信号=-1
日期: 2020-03-13 当前信号=-1
日期: 2020-03-13 当前信号=-1
日期: 2020-03-13 当前信号=-1
日期: 2020-03-13 当前信号=-1
日期: 2020-03-13 当前信号=-1
日期: 2020-03-13 当前信号=-1
日期: 2020-03-13 当前信号=-1
日期: 2020-03-13 当前信号=-1
日期: 2020-03-13 当前信号=-1
日期: 2020-03-13 当前信号=-1
日期: 2020-03-13 当前信号=-1
日期: 2020-03-13 当前信号=-1
日期: 2020-03-13 当前信号=-1
日期: 2020-03-13 当前信号=-1
日期: 2020-03-13 当前信号=-1
日期: 2020-03-13 当前信号=-1
日期: 2020-03-13 当前信号=-1
日期: 2020-0

2025-11-03 00:10:28,029 [INFO] 2020-02-28:   收益率: -1.06%
2025-11-03 00:10:28,030 [INFO] 2020-02-28: 📊 平仓: 毛利润=-180.00, 净利润=-190.43, 收益率=-1.06%
2025-11-03 00:10:28,030 [INFO] 2020-02-28: 🔄 状态已重置: entry_price=0, direction=None
2025-11-03 00:10:28,031 [INFO] 2020-02-28: 发出做多订单: 价格=10430.00
2025-11-03 00:10:28,032 [INFO] 2020-03-02: 🔍 订单完成检查: size=1, position=1
2025-11-03 00:10:28,032 [INFO] 2020-03-02: ✅ 多头开仓完成: 价格=10405.00, entry_bar=205, entry_price=10405.00
2025-11-03 00:10:28,033 [INFO] 2020-03-02: 📋 买入开多仓完成: 价格=10405.00, 成本=10405.00, 手续费=5.20
2025-11-03 00:10:28,033 [INFO] 2020-03-02: 多头平仓，周趋势反转
2025-11-03 00:10:28,034 [INFO] 2020-03-03: 🔍 订单完成检查: size=-1, position=None
2025-11-03 00:10:28,034 [INFO] 2020-03-03: ⚠️ 跳过开仓设置: position_direction=多头, entry_price=10405.0
2025-11-03 00:10:28,035 [INFO] 2020-03-03: 📋 卖出平多仓完成: 价格=10555.00, 成本=10405.00, 手续费=5.28
2025-11-03 00:10:28,035 [INFO] 2020-03-03:   系统毛利润: 150.00
2025-11-03 00:10:28,035 [INFO] 2020-03-03:   系统手续费: 10.48
2025-11-03 00:10


📈 策略绩效报告
初始资金: 100,000.00
最终资产: 101,416.03
总收益率: 1.42%
交易次数: 218
盈利次数: 113
胜率: 51.83%
总盈亏: 4,015.00
交易记录已保存至: triple_screen_trades_红枣连续_20251031.csv


<IPython.core.display.Javascript object>

最终资金: 101416.03
